In [ ]:
import pandas as pd
import json
import datetime
import time
import os
import numpy as np
from config import DATA_BSTS_STATS_FILE, INTERVENTION_CALENDAR, DATA_VACATIONS_INTERVENTION_FILE, DATA_CONSUMPTION_PROCESSED_FILE, DATA_WEATHER_PROCESSED_FILE,  DATA_METADATA_PROCESSED_FILE, DATA_HOLIDAYS_PROCESSED_FILE, DATA_ISO_CONSUMPTION_PROCESSED_FILE, DATA_ENTHALPY_GRADIENTS_PROCESSED_FILE, DATA_SOLAR_GAINS_PROCESSED_FILE, DATA_GBM_CONSUMPTION_PROCESSED_FILE_AGGREGATED
import matplotlib
import matplotlib.pyplot as plt
import os
import sys
from causalimpact import CausalImpact
import warnings
from auxiliary import graph_check_all_bsts, graph_check_all_bsts_control, week_of_month,prepare_data_synthetic_bsts, graph_check_all_bsts_table, graph_point_effects
warnings.filterwarnings('ignore')

In [ ]:
data_mean = pd.read_csv(DATA_GBM_CONSUMPTION_PROCESSED_FILE_AGGREGATED)

In [ ]:
def prepare_data_control(data, INTERVENTION):
    #let's get the data for the first experiment of sensors in VIEW
        #let's get the data for the first experiment of sensors in VIEW
    df = data.copy()
    data_selection = df[df['INTERVENTION']== INTERVENTION]
    data_control = df[df['INTERVENTION']== list(INTERVENTION)[0]+'CONTROL']
    data_mean = data_selection.merge(data_control, left_index=True, right_index=True, suffixes=('', '_y'))
    dict_info = {'y':data_mean['consumption_kWh'], 
                 'x1':data_mean['consumption_kWh_y'],}
    data = pd.DataFrame(dict_info)
    return data

In [ ]:
#check graph
def graph_bsts(ci, end_intervention_date):
    font = {'family' : 'Arial',
            'size'   : 14}
    ax = ci.plot(figsize = (7,7), end_intervention_date=end_intervention_date)
    matplotlib.rc('font', **font)
    return ax

In [ ]:
#MAKE GRAPH COMPARING WITH CONTROL GROUP
%load_ext autoreload
%autoreload 2
X_names = ['GBM_consumption_kWh_0.50','GBM_consumption_kWh_0.975','GBM_consumption_kWh_0.025']
ci, data_synth = graph_check_all_bsts(data_mean, '1T1', X_names, title= "(a) Treatment T1")
ci, data_control = graph_check_all_bsts_control(data_mean, '1T1', X_names, title= "(a) Treatment T1")

In [ ]:
data = data_synth.merge(data_control, left_index=True, right_index=True)
data = data.set_index(pd.to_datetime(data.index))
visual = data[['Observation_x', 'Bayes. Synth. Control Group', 'Random. Control Group']]
font = {'family' : 'Arial',
        'size'   : 18}
ax = visual[:60].plot()
ax.set_xlabel("")
ax.set_ylabel('Daily Consumption [kWh]')
ax.legend(['Observation', 'Bayes. Synth, Control Group','Random. Control Group']);
matplotlib.rc('font', **font)
plt.rcParams["figure.figsize"] = (6,4)

# INTERVENTION 1T1

In [ ]:
%load_ext autoreload
%autoreload 2
X_names = ['GBM_consumption_kWh_0.50','GBM_consumption_kWh_0.975','GBM_consumption_kWh_0.025']
ci = graph_check_all_bsts(data_mean, '1T1', X_names, title= "(a) Treatment T1")

In [ ]:
ci =graph_check_all_bsts(data_mean, '1T2', X_names, title= "(b) Treatment T2")
print(ci.summary_data)
print(ci.summary())
print(ci.summary('report'))

In [ ]:
graph_check_all_bsts(data_mean, '2T4', X_names, title= "(a) Treatment T3")

In [ ]:
graph_check_all_bsts(data_mean, '2T5', X_names, title= "(b) Treatment T4")

In [ ]:
ci = graph_check_all_bsts(data_mean, '3T1', X_names, title= "(b) Treatment T2'")
print(ci.summary('report'))
print(ci.summary())

In [ ]:
ci = graph_check_all_bsts(data_mean, '3T2', X_names, title= "(a) Treatment T1'")
print(ci.summary('report'))
print(ci.summary())

# calculate summary table

In [ ]:
%load_ext autoreload
%autoreload 2
X_names = ['GBM_consumption_kWh_0.50','GBM_consumption_kWh_0.975','GBM_consumption_kWh_0.025']
interventions = ['1T1', '1T2', '2T4', '2T5', '3T2', '3T1']
samples = [16, 18, 22, 18, 16, 18]
final_data = pd.DataFrame()
for intervention, n in zip(interventions, samples):
    final_data = final_data.append(graph_check_all_bsts_table(data_mean, intervention, X_names, n=n),ignore_index=True)

In [ ]:
final_data

In [ ]:
final_data.to_csv(DATA_BSTS_STATS_FILE)

# calculate significance among groups

In [ ]:
%load_ext autoreload
%autoreload 2
X_names = ['GBM_consumption_kWh_0.50','GBM_consumption_kWh_0.975','GBM_consumption_kWh_0.025']
interventions = ['1T1', '1T2', '2T4', '2T5', '3T2', '3T1']
interventions_name = ['T1', 'T2', 'T3', 'T4', "T1'", "T2'"]
samples = [16, 18, 22, 18, 16, 18]
final_data = {}
for intervention, name in zip(interventions,interventions_name):
    final_data[name] = graph_point_effects(data_mean, intervention, X_names)

In [ ]:
import numpy as np
from scipy.stats import mannwhitneyu
memory = []
for d1 in final_data.keys():
    for d2 in final_data.keys():
        if d1 != d2:
            if ((d1, d2) not in memory) and ((d2, d1) not in memory):
                memory.append((d1, d2))
                stat, p = mannwhitneyu(final_data[d1], final_data[d2])
                print('Results for {} and {} : p={:.3f}'.format(d1, d2, p))

In [ ]:
memory